In [1]:
import pandas as pd 
import numpy as np
import xml.etree.ElementTree as ET

tree = ET.parse('XMLs\\periode_01.xml')#-----------------------------------------------
root = tree.getroot()

base_df=pd.read_csv('Data_pl\\base_table_01.csv',index_col='Unnamed: 0')
base_df['Lieferfrist']=base_df['Lieferfrist'].str.replace(',','.').astype('float')
base_df['Abweichung']=base_df['Abweichung'].str.replace(',','.').astype('float')

dfl=[]
for a in root.find('warehousestock').findall('article'):
    dfl.append(a.attrib)
bestands_df = pd.DataFrame(dfl) 

for col in bestands_df.columns:
    try:
        bestands_df[col]=bestands_df[col].astype(int)
    except:
        bestands_df[col]=bestands_df[col].str.replace(',','.')
        bestands_df[col]=bestands_df[col].astype(float)

dfl=[]
for a in root.find('futureinwardstockmovement').findall('order'):
    dfl.append(a.attrib)
futur_orders_df = pd.DataFrame(dfl)

for col in futur_orders_df.columns:
    futur_orders_df[col]=futur_orders_df[col].astype(int)

dfl=[]
for a in root.find('inwardstockmovement').findall('order'):
    dfl.append(a.attrib)
orders_in_df = pd.DataFrame(dfl)

for col in orders_in_df.columns:
    try:
        orders_in_df[col]=orders_in_df[col].astype(int)
    except:
        orders_in_df[col]=orders_in_df[col].str.replace(',','.')
        orders_in_df[col]=orders_in_df[col].astype(float)

In [16]:
periode=2 #-----------------------------------------------------
periodes=list(range(periode,periode+4))

plan=[#--------------------------------------------------------
['P1',150,150,150,150],
['P2',100,100,100,100],
['P3',100,50,50,50]]

plan=pd.DataFrame(plan,columns=['Pr']+periodes)
plan.index=plan['Pr']
plan.drop(columns=['Pr'],inplace=True)
plan

,3,4,5,6
Pr,,,,
P1,150,150,150,150
P2,100,100,100,100
P3,100,50,50,50


In [17]:
merge_1=pd.merge(base_df,bestands_df,left_on='NR. K',right_on='id').drop(columns=['id','startamount','pct','price','stockvalue'])

for col in periodes:
    merge_1[f'Bedarf_{col}']=merge_1[['P1','P2','P3']]@plan[col]

In [18]:
merge_2=pd.merge(futur_orders_df,base_df,left_on='article',right_on='NR. K')
merge_2['avg_order_time']=merge_2['Lieferfrist']+merge_2['Abweichung']
merge_2=merge_2.drop(columns=['id','NR. K','P1','P2','P3','Diskontmenge','Lieferfrist','Abweichung'])

merge_2['avg_order_time']=merge_2.apply(lambda x : x['avg_order_time'] if x['mode'] == 5 else x['avg_order_time']/2,axis=1)
merge_2['expected_arival_periode']=merge_2['orderperiod']+merge_2['avg_order_time']
merge_2['expected_arival_periode']=np.floor(merge_2['expected_arival_periode'])

In [19]:
# current_col=5
def order_sum_at_periode(x):
    global current_col
    return sum(merge_2[(merge_2['article']==x['NR. K'])&(merge_2['expected_arival_periode']==current_col)]['amount'])

In [20]:
amount_df=merge_1.copy()
current_col=periodes[0]

amount_df[(f'paa_{periodes[0]}','start')]=amount_df['amount']
amount_df[(f'paa_{periodes[0]}','bed')]=amount_df[f'Bedarf_{periodes[0]}']
amount_df[(f'paa_{periodes[0]}','o_in')]=amount_df.apply(order_sum_at_periode,axis=1)

current_col=periodes[0]-1
amount_df[(f'paa_{periodes[0]}','o_in')]+=amount_df.apply(order_sum_at_periode,axis=1)

amount_df[(f'paa_{periodes[0]}','end')]=(amount_df['amount']-amount_df[f'Bedarf_{periodes[0]}']).apply(lambda x : 0 if x<=0 else x)
amount_df[(f'paa_{periodes[0]}','end')]+=amount_df[(f'paa_{periodes[0]}','o_in')]


for col in periodes[1:]:
    current_col=col
    amount_df[(f'paa_{col}','start')]=amount_df[(f'paa_{col-1}','end')]
    amount_df[(f'paa_{col}','bed')]=amount_df[f'Bedarf_{col}']
    amount_df[(f'paa_{col}','o_in')]=amount_df.apply(order_sum_at_periode,axis=1) 
    amount_df[(f'paa_{col}','end')]=(amount_df[(f'paa_{col-1}','end')]-amount_df[f'Bedarf_{col}']).apply(lambda x : 0 if x<=0 else x)
    amount_df[(f'paa_{col}','end')]+=amount_df[(f'paa_{col}','o_in')]


In [21]:
l=[]
for i,x in enumerate(amount_df.columns):
    if type(x)!=tuple:
        l.append((x,''))
    else:
        l.append(x)

amount_df.columns=l
amount_df.columns=pd.MultiIndex.from_tuples(amount_df.columns)

In [22]:
merge_2

,orderperiod,mode,article,amount,avg_order_time,expected_arival_periode
0,1,4,32,2700,1.30,2.0
1,1,4,35,3600,1.30,2.0
2,1,5,47,900,1.20,2.0
3,1,5,48,1800,1.20,2.0
4,1,5,59,1800,0.90,1.0
5,1,5,36,900,1.30,2.0
6,1,5,42,1800,1.50,2.0
7,1,5,44,1200,1.20,2.0
8,1,5,46,900,1.20,2.0
9,1,5,53,22000,1.80,2.0


In [23]:
out=amount_df[[f'paa_{x}' for x in periodes]]
out.set_index(amount_df['NR. K'])

paa_3                       paa_4                     paa_5         \
       start    bed   o_in    end  start    bed o_in    end  start    bed   
NR. K                                                                       
21       150    150    300    300    300    150    0    150    150    150   
22       200    100    300    400    400    100    0    300    300    100   
23       200    100      0    100    100     50    0     50     50     50   
24      3650   2450   6100   7300   7300   2100    0   5200   5200   2100   
25      5800   1400      0   4400   4400   1200    0   3200   3200   1200   
27      2900    700      0   2200   2200    600    0   1600   1600    600   
28      7300   1700      0   5600   5600   1400    0   4200   4200   1400   
32      1840   1050   2700   3490   3490    900    0   2590   2590    900   
33       700    200      0    500    500    100    0    400    400    100   
34     14800   7200      0   7600   7600   3600    0   4000   4000   3600   
35      2280   1400   3600   4480   4480   1200    0   3280   3280   1200   
36       550    350    900   1100   1100    300    0    800    800    300   
37       590    350    900   1140   1140    300    0    840    840    300   
38       990    350      0    640    640    300    0    340    340    300   
39      2000    700      0   1300   1300    600    0    700    700    600   
40       550    350    900   1100   1100    300    0    800    800    300   
41      1450    350      0   1100   1100    300    0    800    800    300   
42      1100    700   1800   2200   2200    600    0   1600   1600    600   
43      1550    350      0   1200   1200    300    0    900    900    300   
44      1750   1050   1200   1900   1900    900    0   1000   1000    900   
45      1450    350      0   1100   1100    300    0    800    800    300   
46       550    350    900   1100   1100    300    0    800    800    300   
47       630    350    900   1180   1180    300    0    880    880    300   
48      1200    700   1800   2300   2300    600    0   1700   1700    600   
52       900    300      0    600    600    300    0    300    300    300   
53     11200  10800  22000  22400  22400  10800    0  11600  11600  10800   
57      1000    200      0    800    800    200    0    600    600    200   
58     36800   7400      0  29400  29400   7300    0  22100  22100   7300   
59      1480    700      0    780    780    600    0    180    180    600   

                   paa_6                    
      o_in    end  start    bed o_in   end  
NR. K                                       
21       0      0      0    150    0     0  
22       0    200    200    100    0   100  
23       0      0      0     50    0     0  
24       0   3100   3100   2100    0  1000  
25       0   2000   2000   1200    0   800  
27       0   1000   1000    600    0   400  
28       0   2800   2800   1400    0  1400  
32       0   1690   1690    900    0   790  
33       0    300    300    100    0   200  
34       0    400    400   3600    0     0  
35       0   2080   2080   1200    0   880  
36       0    500    500    300    0   200  
37       0    540    540    300    0   240  
38       0     40     40    300    0     0  
39       0    100    100    600    0     0  
40       0    500    500    300    0   200  
41       0    500    500    300    0   200  
42       0   1000   1000    600    0   400  
43       0    600    600    300    0   300  
44       0    100    100    900    0     0  
45       0    500    500    300    0   200  
46       0    500    500    300    0   200  
47       0    580    580    300    0   280  
48       0   1100   1100    600    0   500  
52       0      0      0    300    0     0  
53       0    800    800  10800    0     0  
57       0    400    400    200    0   200  
58       0  14800  14800   7300    0  7500  
59       0      0      0    600    0     0

In [24]:
def zero_amount_period(x):
    bestand=x['amount']
    zero_amount_p=np.inf

    for i in periodes:
        bestand-=x[f'paa_{i}']
        if bestand<=0:
            zero_amount_p=i+1
            break
    return zero_amount_p

In [25]:
#amount_df['0_amount_p']=amount_df.apply(zero_amount_period,axis=1)

In [26]:
merge_2

,orderperiod,mode,article,amount,avg_order_time,expected_arival_periode
0,1,4,32,2700,1.30,2.0
1,1,4,35,3600,1.30,2.0
2,1,5,47,900,1.20,2.0
3,1,5,48,1800,1.20,2.0
4,1,5,59,1800,0.90,1.0
5,1,5,36,900,1.30,2.0
6,1,5,42,1800,1.50,2.0
7,1,5,44,1200,1.20,2.0
8,1,5,46,900,1.20,2.0
9,1,5,53,22000,1.80,2.0
